In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

In [4]:
# NBA Data Files
games = "Resources/games.csv"
games_details = "Resources/games_details.csv"
players = "Resources/players.csv"
ranking = "Resources/ranking.csv"
teams = "Resources/teams.csv"

In [6]:
# Read the data from games dataset
games_df = pd.read_csv(games)
print(games_df.shape)
pd.set_option('display.max_columns',50)
games_df.tail()

(26651, 21)


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
26646,2014-10-06,11400007,Final,1610612737,1610612740,2014,1610612737,93.0,0.419,0.821,0.421,24.0,50.0,1610612740,87.0,0.366,0.643,0.375,17.0,43.0,1
26647,2014-10-06,11400004,Final,1610612741,1610612764,2014,1610612741,81.0,0.338,0.719,0.381,18.0,40.0,1610612764,85.0,0.411,0.636,0.267,17.0,47.0,0
26648,2014-10-06,11400005,Final,1610612747,1610612743,2014,1610612747,98.0,0.448,0.682,0.500,29.0,45.0,1610612743,95.0,0.387,0.659,0.500,19.0,43.0,1
26649,2014-10-05,11400002,Final,1610612761,1610612758,2014,1610612761,99.0,0.440,0.771,0.333,21.0,30.0,1610612758,94.0,0.469,0.725,0.385,18.0,45.0,1
26650,2014-10-04,11400001,Final,1610612748,1610612740,2014,1610612748,86.0,0.431,0.679,0.333,18.0,42.0,1610612740,98.0,0.462,0.706,0.438,19.0,42.0,0


In [11]:
# Checking the date range of this data
# First finding the minimum year of this data
min_yr = games_df["SEASON"].min()

# Finding the maximum year of the data
max_yr = games_df["SEASON"].max()
print(f' This data is for NBA seasons from {min_yr} to {max_yr}')

 This data is for NBA seasons from 2003 to 2022


In [57]:
# How many games do I have data from?
games_df['GAME_ID'].nunique()

26622

In [15]:
# Looking at column names and data types to get a sense of the data that is present in this games dataset
games_df.dtypes

GAME_DATE_EST        object
GAME_ID               int64
GAME_STATUS_TEXT     object
HOME_TEAM_ID          int64
VISITOR_TEAM_ID       int64
SEASON                int64
TEAM_ID_home          int64
PTS_home            float64
FG_PCT_home         float64
FT_PCT_home         float64
FG3_PCT_home        float64
AST_home            float64
REB_home            float64
TEAM_ID_away          int64
PTS_away            float64
FG_PCT_away         float64
FT_PCT_away         float64
FG3_PCT_away        float64
AST_away            float64
REB_away            float64
HOME_TEAM_WINS        int64
dtype: object

In [28]:
# Looking at unique counts to better understand this data
unique_counts = games_df.nunique()
print(unique_counts)

GAME_DATE_EST        4304
GAME_ID             26622
GAME_STATUS_TEXT        1
HOME_TEAM_ID           30
VISITOR_TEAM_ID        30
SEASON                 20
TEAM_ID_home           30
PTS_home              100
FG_PCT_home           404
FT_PCT_home           410
FG3_PCT_home          437
AST_home               42
REB_home               55
TEAM_ID_away           30
PTS_away              106
FG_PCT_away           393
FT_PCT_away           387
FG3_PCT_away          417
AST_away               41
REB_away               52
HOME_TEAM_WINS          2
dtype: int64


In [16]:
# Read the data from games_details dataset
games_details_df = pd.read_csv(games_details)
print(games_details_df.shape)
games_details_df.head()

(668628, 29)


/var/folders/k1/p7f99z1557z1twq8wpyj8m0r0000gn/T/ipykernel_38867/3967383474.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  games_details_df = pd.read_csv(games_details)


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,Romeo,F,NaN,18:06,1.0,1.0,1.000,0.0,0.0,0.000,0.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,Jeremy,F,NaN,31:01,7.0,14.0,0.500,2.0,4.0,0.500,7.0,10.0,0.7,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,21:42,6.0,9.0,0.667,0.0,0.0,0.000,1.0,1.0,1.0,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,NaN,30:20,4.0,13.0,0.308,1.0,6.0,0.167,1.0,1.0,1.0,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,Tre,G,NaN,27:44,7.0,12.0,0.583,1.0,3.0,0.333,4.0,4.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0


In [33]:
# Finding the columns with mixed datatypes
for col in games_details_df:
    unique_types = games_details_df[col].apply(type).unique()
    if len(unique_types) > 1:
        print(col, unique_types)

NICKNAME [<class 'str'> <class 'float'>]
START_POSITION [<class 'str'> <class 'float'>]
COMMENT [<class 'float'> <class 'str'>]
MIN [<class 'str'> <class 'float'>]


In [17]:
# Looking at column names and data types to get a sense of the data that is present in this games_details dataset
games_details_df.dtypes

GAME_ID                int64
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_CITY             object
PLAYER_ID              int64
PLAYER_NAME           object
NICKNAME              object
START_POSITION        object
COMMENT               object
MIN                   object
FGM                  float64
FGA                  float64
FG_PCT               float64
FG3M                 float64
FG3A                 float64
FG3_PCT              float64
FTM                  float64
FTA                  float64
FT_PCT               float64
OREB                 float64
DREB                 float64
REB                  float64
AST                  float64
STL                  float64
BLK                  float64
TO                   float64
PF                   float64
PTS                  float64
PLUS_MINUS           float64
dtype: object

In [34]:
# Read the data from players dataset
players_df = pd.read_csv(players)
print(players_df.shape)
players_df.head()

(7228, 4)


,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,Royce O'Neale,1610612762,1626220,2019
1,Bojan Bogdanovic,1610612762,202711,2019
2,Rudy Gobert,1610612762,203497,2019
3,Donovan Mitchell,1610612762,1628378,2019
4,Mike Conley,1610612762,201144,2019


In [36]:
# Checking that the player IDs are consistent across dataframes

royce = 1626220
royce_check = games_details_df.loc[games_details_df['PLAYER_ID']== royce]
royce_check

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
165,22200470,1610612751,BKN,Brooklyn,1626220,Royce O'Neale,Royce,F,NaN,26:50,5.0,7.0,0.714,4.0,6.0,0.667,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,3.0,2.0,14.0,28.0
723,22200447,1610612751,BKN,Brooklyn,1626220,Royce O'Neale,Royce,F,NaN,31:37,4.0,6.0,0.667,3.0,4.0,0.750,0.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,1.0,3.0,11.0,-2.0
1147,22200433,1610612751,BKN,Brooklyn,1626220,Royce O'Neale,Royce,F,NaN,32.000000:29,0.0,4.0,0.000,0.0,4.0,0.000,0.0,0.0,0.0,0.0,7.0,7.0,3.0,0.0,0.0,3.0,5.0,0.0,-4.0
2385,22200380,1610612751,BKN,Brooklyn,1626220,Royce O'Neale,Royce,F,NaN,37:19,2.0,5.0,0.400,2.0,4.0,0.500,0.0,2.0,0.0,3.0,6.0,9.0,5.0,0.0,1.0,3.0,1.0,6.0,7.0
2658,22200364,1610612751,BKN,Brooklyn,1626220,Royce O'Neale,Royce,F,NaN,37:22,1.0,6.0,0.167,1.0,4.0,0.250,0.0,0.0,0.0,2.0,2.0,4.0,4.0,1.0,0.0,1.0,4.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481478,21700023,1610612762,UTA,Utah,1626220,Royce O'Neale,NaN,NaN,DNP - Coach's Decision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481825,21700010,1610612762,UTA,Utah,1626220,Royce O'Neale,NaN,NaN,DNP - Coach's Decision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
482834,11700058,1610612762,UTA,Utah,1626220,Royce O'Neale,NaN,NaN,NaN,9:23,0.0,1.0,0.000,0.0,1.0,0.000,0.0,0.0,0.0,0.0,3.0,3.0,1.0,1.0,1.0,1.0,2.0,0.0,6.0
482914,11700047,1610612762,UTA,Utah,1626220,Royce O'Neale,NaN,NaN,NaN,6:53,2.0,4.0,0.500,1.0,3.0,0.333,0.0,0.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,5.0,-2.0


In [37]:
# Read the data from ranking dataset
ranking_df = pd.read_csv(ranking)
print(ranking_df.shape)
ranking_df.head()

(210342, 13)


,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,NaN
1,1610612763,0,22022,2022-12-22,West,Memphis,30,19,11,0.633,13-2,6-9,NaN
2,1610612740,0,22022,2022-12-22,West,New Orleans,31,19,12,0.613,13-4,6-8,NaN
3,1610612756,0,22022,2022-12-22,West,Phoenix,32,19,13,0.594,14-4,5-9,NaN
4,1610612746,0,22022,2022-12-22,West,LA Clippers,33,19,14,0.576,11-7,8-7,NaN


In [38]:
# Read the data from teams dataset
teams_df = pd.read_csv(teams)
print(teams_df.shape)
teams_df.head()

(30, 14)


,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends


In [39]:
# Create a new dataframe and merge both game datasets
game_data_merge_df = pd.merge(games_details_df, games_df, on='GAME_ID')
print(game_data_merge_df.shape)
pd.set_option('display.max_columns',50)
display(game_data_merge_df)

(669560, 49)


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,GAME_DATE_EST,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,Romeo,F,NaN,18:06,1.0,1.0,1.000,0.0,0.0,0.000,0.0,0.0,0.000,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0,2022-12-22,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,0.382,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,Jeremy,F,NaN,31:01,7.0,14.0,0.500,2.0,4.0,0.500,7.0,10.0,0.700,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0,2022-12-22,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,0.382,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
2,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,21:42,6.0,9.0,0.667,0.0,0.0,0.000,1.0,1.0,1.000,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0,2022-12-22,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,0.382,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
3,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,NaN,30:20,4.0,13.0,0.308,1.0,6.0,0.167,1.0,1.0,1.000,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0,2022-12-22,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,0.382,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
4,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,Tre,G,NaN,27:44,7.0,12.0,0.583,1.0,3.0,0.333,4.0,4.0,1.000,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0,2022-12-22,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,0.382,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669555,11200005,1610612743,DEN,Denver,202706,Jordan Hamilton,NaN,NaN,NaN,19,4.0,9.0,0.444,3.0,6.0,0.500,6.0,7.0,0.857,0.0,2.0,2.0,0.0,2.0,0.0,1.0,3.0,17.0,NaN,2012-10-06,Final,1610612746,1610612743,2012,1610612746,104.0,0.512,0.700,0.267,21.0,40.0,1610612743,106.0,0.532,0.615,0.471,25.0,34.0,0
669556,11200005,1610612743,DEN,Denver,202702,Kenneth Faried,NaN,NaN,NaN,23,7.0,11.0,0.636,0.0,0.0,0.000,4.0,4.0,1.000,1.0,0.0,1.0,1.0,1.0,0.0,3.0,3.0,18.0,NaN,2012-10-06,Final,1610612746,1610612743,2012,1610612746,104.0,0.512,0.700,0.267,21.0,40.0,1610612743,106.0,0.532,0.615,0.471,25.0,34.0,0
669557,11200005,1610612743,DEN,Denver,201585,Kosta Koufos,NaN,NaN,NaN,15,3.0,7.0,0.429,0.0,0.0,0.000,0.0,0.0,0.000,3.0,5.0,8.0,0.0,1.0,0.0,0.0,3.0,6.0,NaN,2012-10-06,Final,1610612746,1610612743,2012,1610612746,104.0,0.512,0.700,0.267,21.0,40.0,1610612743,106.0,0.532,0.615,0.471,25.0,34.0,0
669558,11200005,1610612743,DEN,Denver,202389,Timofey Mozgov,NaN,NaN,NaN,19,1.0,1.0,1.000,0.0,0.0,0.000,0.0,2.0,0.000,1.0,2.0,3.0,1.0,0.0,0.0,4.0,2.0,2.0,NaN,2012-10-06,Final,1610612746,1610612743,2012,1610612746,104.0,0.512,0.700,0.267,21.0,40.0,1610612743,106.0,0.532,0.615,0.471,25.0,34.0,0


In [41]:
# Columns list for new df

game_data_merge_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS', 'GAME_DATE_EST', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID',
       'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away', 'HOME_TEAM_WINS'],
      dtype='object')

In [40]:
# Checking that the player stats within a game are equal to the games stats from games_df

# Setting up my aggregations to ensure that each column is aggregated appropriately
# aggregations = {
    'TEAM_ABBREVIATION':'first',
    'TEAM_CITY':'first',
    'PLAYER_NAME':'first',
    'NICKNAME':'first',
    'START_POSITION':'first',
    'COMMENT':'first',
    'MIN',
    'FGM',
    'FGA',
    'FG_PCT',
    'FG3M',
    'FG3A',
    'FG3_PCT',
    'FTM',
    'FTA',
    'FT_PCT',
    'OREB',
    'DREB',
    'REB',
    'AST',
    'STL',
    'BLK',
    'TO',
    'PF',
    'PTS',
    'PLUS_MINUS',
    'GAME_DATE_EST',
    'GAME_STATUS_TEXT',
    'HOME_TEAM_ID',
    'VISITOR_TEAM_ID',
    'SEASON', 'TEAM_ID_home',
    'PTS_home', 'FG_PCT_home',
    'FT_PCT_home',
    'FG3_PCT_home',
    'AST_home',
    'REB_home',
    'TEAM_ID_away',
    'PTS_away',
    'FG_PCT_away',
    'FT_PCT_away',
    'FG3_PCT_away',
    'AST_away',
    'REB_away',
    'HOME_TEAM_WINS'
}

# games_total_stats = game_data_merge_df.groupby(['GAME_ID', 'TEAM_ID']).agg()

PLAYER_ID   FGM   FGA  FG_PCT  FG3M  FG3A  FG3_PCT   FTM  \
GAME_ID  TEAM_ID                                                                
10300001 1610612742      17609  34.0  76.0   4.177   2.0   8.0    1.000  15.0   
         1610612762      38429  32.0  70.0   9.758   1.0   7.0    1.000  25.0   
10300002 1610612749      23082  32.0  75.0   4.521   2.0  13.0    0.500  28.0   
         1610612763      32790  40.0  81.0   6.789   4.0  15.0    1.367  21.0   
10300003 1610612739      26307  38.0  77.0   5.051   4.0   6.0    1.500  20.0   
...                        ...   ...   ...     ...   ...   ...      ...   ...   
52100131 1610612759   22807688  37.0  91.0   3.730  13.0  34.0    2.238  16.0   
52100201 1610612737   15474550  39.0  80.0   4.813   9.0  31.0    1.697  20.0   
         1610612739   16895340  37.0  81.0   3.973  13.0  36.0    1.920  14.0   
52100211 1610612740   15472404  43.0  88.0   4.354   6.0  22.0    1.143  13.0   
         1610612746   10786535  35.0  85.0   3.948   9.0  36.0    1.336  22.0   

                      FTA  FT_PCT  OREB  DREB   REB   AST   STL  BLK    TO  \
GAME_ID  TEAM_ID                                                             
10300001 1610612742  30.0   3.191  12.0  26.0  38.0  20.0   9.0  4.0  18.0   
         1610612762  34.0   8.183   9.0  32.0  41.0  23.0   8.0  4.0  18.0   
10300002 1610612749  40.0   6.392  11.0  32.0  43.0  20.0   9.0  4.0  24.0   
         1610612763  34.0   5.306  14.0  34.0  48.0  25.0  18.0  7.0  25.0   
10300003 1610612739  29.0   7.000  12.0  40.0  52.0  25.0  10.0  7.0  25.0   
...                   ...     ...   ...   ...   ...   ...   ...  ...   ...   
52100131 1610612759  22.0   5.100   8.0  26.0  34.0  19.0   4.0  3.0   6.0   
52100201 1610612737  22.0   4.389  10.0  32.0  42.0  15.0   5.0  3.0  13.0   
         1610612739  17.0   4.250   8.0  28.0  36.0  23.0   8.0  4.0  11.0   
52100211 1610612740  25.0   4.500  17.0  37.0  54.0  20.0   2.0  3.0   8.0   
         1610612746  34.0   3.363  12.0  33.0  45.0  20.0   7.0  3.0   7.0   

                       PF    PTS  PLUS_MINUS  HOME_TEAM_ID  VISITOR_TEAM_ID  \
GAME_ID  TEAM_ID                                                              
10300001 1610612742  34.0   85.0         0.0   17716740382      17716740162   
         1610612762  26.0   90.0         0.0   30601642478      30601642098   
10300002 1610612749  26.0   94.0         0.0   20937965919      20937965737   
         1610612763  35.0  105.0         0.0   24159191445      24159191235   
10300003 1610612739  33.0  100.0         0.0   22548578710      22548578346   
...                   ...    ...         ...           ...              ...   
52100131 1610612759  20.0  103.0       -50.0   22548578360      22548578626   
52100201 1610612737  19.0  107.0        30.0   20937965607      20937965581   
         1610612739  20.0  101.0       -30.0   20937965607      20937965581   
52100211 1610612740  26.0  105.0        20.0   20937965698      20937965620   
         1610612746  21.0  101.0       -20.0   17716740206      17716740140   

                     SEASON  TEAM_ID_home  PTS_home  FG_PCT_home  FT_PCT_home  \
GAME_ID  TEAM_ID                                                                
10300001 1610612742   22033   17716740382     990.0        5.027        8.085   
         1610612762   38057   30601642478    1710.0        8.683       13.965   
10300002 1610612749   26039   20937965919    1365.0        6.422        8.034   
         1610612763   30045   24159191445    1575.0        7.410        9.270   
10300003 1610612739   28042   22548578710    1344.0        5.474        8.400   
...                     ...           ...       ...          ...          ...   
52100131 1610612759   28294   22548578360    1582.0        7.602       10.346   
52100201 1610612737   26273   20937965607    1313.0        5.941       10.712   
         1610612739   26273   20937965607    1313.0        5.941       10.712   
52100211 1610612740   26273   209

In [42]:
# Create a new dataframe and merge the team and rankings data sets
team_and_rankings_df = pd.merge(teams_df, ranking_df, on='TEAM_ID')
print(team_and_rankings_df.shape)
pd.set_option('display.max_columns',50)
display(team_and_rankings_df)

(210342, 26)


,LEAGUE_ID_x,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION,LEAGUE_ID_y,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks,0,22022,2022-12-22,East,Atlanta,32,16,16,0.500,10-6,6-10,NaN
1,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks,0,22022,2022-12-21,East,Atlanta,32,16,16,0.500,10-6,6-10,NaN
2,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks,0,22022,2022-12-20,East,Atlanta,31,16,15,0.516,10-5,6-10,NaN
3,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks,0,22022,2022-12-19,East,Atlanta,31,16,15,0.516,10-5,6-10,NaN
4,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks,0,22022,2022-12-18,East,Atlanta,30,15,15,0.500,9-5,6-10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210337,0,1610612744,1946,2019,GSW,Warriors,1946,Golden State,Chase Center,19596.0,Joe Lacob,Bob Myers,Steve Kerr,Santa Cruz Warriors,0,22013,2014-09-05,West,Golden State,82,51,31,0.622,27-14,24-17,NaN
210338,0,1610612744,1946,2019,GSW,Warriors,1946,Golden State,Chase Center,19596.0,Joe Lacob,Bob Myers,Steve Kerr,Santa Cruz Warriors,0,22013,2014-09-04,West,Golden State,82,51,31,0.622,27-14,24-17,NaN
210339,0,1610612744,1946,2019,GSW,Warriors,1946,Golden State,Chase Center,19596.0,Joe Lacob,Bob Myers,Steve Kerr,Santa Cruz Warriors,0,22013,2014-09-03,West,Golden State,82,51,31,0.622,27-14,24-17,NaN
210340,0,1610612744,1946,2019,GSW,Warriors,1946,Golden State,Chase Center,19596.0,Joe Lacob,Bob Myers,Steve Kerr,Santa Cruz Warriors,0,22013,2014-09-02,West,Golden State,82,51,31,0.622,27-14,24-17,NaN


In [56]:
# Now answering our question about the most consistent players
# Our definition of 'most consistent' is 'has the smallest total standard devation in the dataset for a given stat'
# I will focus on rebounding statistics

rebounds_std = games_details_df.groupby('PLAYER_ID').agg({'REB':'std', 'PLAYER_NAME':'first'}).reset_index()
rebounds_std

# Sort this from smallest to largest
rebounds_std.sort_values(by='REB', ascending=True)

,PLAYER_ID,REB,PLAYER_NAME
2294,1629700,0.0,Kaleb Wesson
294,1969,0.0,Peter Cornell
1605,203965,0.0,Justin Cobbs
2344,1629826,0.0,Norense Odiase
2672,1631347,0.0,Anthony Duruji
...,...,...,...
2682,1962936483,NaN,Jan Vesely
2683,1962936489,NaN,Brian Skinner
2684,1962936495,NaN,Damien Wilkins
2685,1962937755,NaN,Paige Marcus


In [ ]:
# Testing a few different ways to cutoff and clean this up to identify players

# Brainstorming:
# min games played, min minutes played / game, min average rebounds per game?
games_cutoff = 82

# Minutes played for each game included
minutes_cutoff = 18

# Average
reb_avg = 5

In [63]:
# Using variables to limit the players data set and checking the results of the std_dev calculation above for each

games_threshold = games_details_df['PLAYER_ID'].value_counts()
print(games_threshold)

2544       1795
2617       1720
2738       1560
2730       1541
2594       1534
           ... 
201251        1
1629260       1
201252        1
974           1
203198        1
Name: PLAYER_ID, Length: 2687, dtype: int64


In [64]:
# Filter out 'PLAYER_ID' values with less than 82 occurrences
valid_player_ids = games_threshold[games_threshold >= 82].index.tolist()

# Create a new DataFrame with 'PLAYER_ID's that have 82 or more occurrences
filtered_df = games_details_df[games_details_df['PLAYER_ID'].isin(valid_player_ids)]
filtered_df

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,Romeo,F,NaN,18:06,1.0,1.0,1.000,0.0,0.0,0.000,0.0,0.0,0.000,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
2,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,21:42,6.0,9.0,0.667,0.0,0.0,0.000,1.0,1.0,1.000,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,NaN,30:20,4.0,13.0,0.308,1.0,6.0,0.167,1.0,1.0,1.000,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,Tre,G,NaN,27:44,7.0,12.0,0.583,1.0,3.0,0.333,4.0,4.0,1.000,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0
5,22200477,1610612759,SAS,San Antonio,1628380,Zach Collins,Zach,NaN,NaN,18:04,2.0,6.0,0.333,0.0,0.0,0.000,2.0,2.0,1.000,1.0,1.0,2.0,2.0,0.0,0.0,0.0,3.0,6.0,-13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668623,11200005,1610612743,DEN,Denver,202706,Jordan Hamilton,NaN,NaN,NaN,19,4.0,9.0,0.444,3.0,6.0,0.500,6.0,7.0,0.857,0.0,2.0,2.0,0.0,2.0,0.0,1.0,3.0,17.0,NaN
668624,11200005,1610612743,DEN,Denver,202702,Kenneth Faried,NaN,NaN,NaN,23,7.0,11.0,0.636,0.0,0.0,0.000,4.0,4.0,1.000,1.0,0.0,1.0,1.0,1.0,0.0,3.0,3.0,18.0,NaN
668625,11200005,1610612743,DEN,Denver,201585,Kosta Koufos,NaN,NaN,NaN,15,3.0,7.0,0.429,0.0,0.0,0.000,0.0,0.0,0.000,3.0,5.0,8.0,0.0,1.0,0.0,0.0,3.0,6.0,NaN
668626,11200005,1610612743,DEN,Denver,202389,Timofey Mozgov,NaN,NaN,NaN,19,1.0,1.0,1.000,0.0,0.0,0.000,0.0,2.0,0.000,1.0,2.0,3.0,1.0,0.0,0.0,4.0,2.0,2.0,NaN


In [70]:
# Recalculating rebounds std with this limited set of players
rebounds_std_filtered = filtered_df.groupby('PLAYER_ID').agg({'REB':'std', 'PLAYER_NAME':'first'}).reset_index()
rebounds_std_filtered

# Sort this from smallest to largest
ranked_reb = rebounds_std_filtered.sort_values(by='REB', ascending=True)
ranked_reb

,PLAYER_ID,REB,PLAYER_NAME
1280,1629607,0.470162,Jared Harper
858,203116,0.775398,Tyshawn Taylor
417,2766,0.799602,Antonio Burks
962,203912,0.836325,C.J. Wilcox
1006,204014,0.903950,Damjan Rudez
...,...,...,...
1387,1630209,5.084952,Omer Yurtseven
83,948,5.097363,Marcus Camby
744,202355,5.223370,Hassan Whiteside
827,203083,5.319487,Andre Drummond


In [74]:
# Recalculating rebounds std with this limited set of players
points_avg_filtered = filtered_df.groupby('PLAYER_ID').agg({'PTS':'mean', 'PLAYER_NAME':'first'}).reset_index()
points_avg_filtered

# Sort this from smallest to largest
top_performers_list = points_avg_filtered.sort_values(by='PTS', ascending=False).reset_index()
top_performers_list.head(50)

,index,PLAYER_ID,PTS,PLAYER_NAME
0,561,201142,27.074539,Kevin Durant
1,1246,1629029,26.941003,Luka Doncic
2,338,2544,26.834292,LeBron James
3,100,977,26.621463,Kobe Bryant
4,82,947,25.863341,Allen Iverson
5,992,203954,25.607229,Joel Embiid
6,1286,1629627,25.049587,Zion Williamson
7,1244,1629027,25.039548,Trae Young
8,825,203081,24.353081,Damian Lillard
9,663,201939,24.318267,Stephen Curry


In [77]:
top_performers_list.to_csv("Resources/Top_Performers.csv", index=False)